In [ ]:
from pynq import Overlay
from pynq.lib.video import *

overlay = Overlay('color_detection.bit')
#overlay?
#hdmi_in = base.video.hdmi_in
hdmi_out = overlay.video.hdmi_out
print('done')

In [ ]:
mode = VideoMode(1920,1080,32,stride=None)
hdmi_out.configure(mode,PIXEL_RGBA)


#PIXEL_RGB = PixelFormat(24, COLOR_IN_RGB, COLOR_OUT_RGB, "RG24")
#PIXEL_RGBA = PixelFormat(32, COLOR_IN_RGB, COLOR_OUT_RGB, "RA24")
#PIXEL_BGR = PixelFormat(24, COLOR_IN_BGR, COLOR_OUT_BGR, "BG24")
#PIXEL_YCBCR = PixelFormat(24, COLOR_IN_YCBCR, COLOR_OUT_YCBCR, "YU24")
#PIXEL_GRAY = PixelFormat(8, COLOR_IN_YCBCR, COLOR_OUT_GRAY)
print('done')

In [ ]:
import cv2
import numpy as np
#from matplotlib import pyplot as plt
from pynq.lib.video import *

img = cv2.imread('color_hd.png') # always read as BGR
#img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

h, w , d= img.shape

print(h,w)

r, g, b = cv2.split(img)

r = r.astype(np.uint32)
g = g.astype(np.uint32)
b = b.astype(np.uint32)
print('done')

In [ ]:
#rgb = np.zeros([h,w],dtype=np.uint32)
#a = np.ones([h,w],dtype=np.uint32)
#rgb = np.add(r<<24,g<<16,b<<8)
#rgb = np.add(rgb,a)
#print('done')

In [ ]:
from matplotlib import pyplot as plt
from pynq import Xlnk
################
rgb = np.zeros([h,w],dtype=np.uint32)
a = np.zeros([h,w],dtype=np.uint32)
#c = 255*np.ones([h,w],dtype=np.uint32)

rgb = np.add(np.add(a<<24,r<<16),np.add(g<<8,b))

#######################################
xlnk = Xlnk()
input_buffer = xlnk.cma_array(shape=(h,w), dtype=np.uint32)

input_buffer[0:h,0:w] = rgb[0:h,0:w];
#plt.imshow(input_buffer)

In [ ]:
hdmi_out.start()

In [ ]:
hdmi_out.writeframe(input_buffer)

In [ ]:
crtl = hdmi_out.Colour_Filter_0
#help(crtl)

In [ ]:
# if lower limit = 0 , upper limit = 0 then pass through
# else apply limits
crtl.write(0x04,0) # lower limit
crtl.write(0x08,0) # upper limit
crtl.read(0x04)
crtl.read(0x08)

In [ ]:
input_buffer.freebuffer()

In [ ]:
hdmi_out.close()

In [ ]:
import cv2
from matplotlib import pyplot as plt

hue_range = cv2.imread('hue_range.jpg')
hue_range = cv2.cvtColor(hue_range, cv2.COLOR_BGR2RGB)
plt.imshow(hue_range)

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
from time import sleep
import cv2

def pxl(r,g,b):
    sleep(2)
    pxl_clr = np.ones([3,30,3],np.uint8)
    print(r,g,b)
    pxl_clr[:,:,0] = r*pxl_clr[:,:,0]
    pxl_clr[:,:,1] = g*pxl_clr[:,:,1]
    pxl_clr[:,:,2] = b*pxl_clr[:,:,2]
    plt.imshow(pxl_clr)
    val = np.uint8([[[r,g,b]]])
    
    max_val = max(r,g,b)
    min_val = min(r,g,b)
    
    diff = max_val - min_val
    
    if max_val == min_val:
        hue = 0
    elif max_val == r:
        hue = (((g-b)/diff)*60)
    elif max_val == g:
        hue = ((2 + ((b-r)/diff))*60)
    elif max_val == b:
        hue = ((4 + ((r-g)/diff))*60)   
    if hue < 0:
        hue = 360 + hue
    print("hue: ",int(hue))
    return([r,g,b])

r = interact(pxl,r=(0, 255,1),g=(0, 255,2),b=(0, 255,3))


print(r)